# Zero-Shot Movie Genre Classifier

**Name:** Anshul Banda  
**SRN:** PES2UG23CS081  
**Section:** B


# Movie Genre Classifier using Zero-Shot Learning

In this project, we build a simple movie genre classifier that predicts the genre of a film based on its plot summary. The classifier assigns one of three possible genres: **Horror**, **Comedy**, or **Action**.

Instead of training a custom model, this project uses **zero-shot classification**, which allows a pre-trained language model to categorize text into labels it has never been explicitly trained on. This approach is useful when labeled training data is limited or unavailable.

By providing the model with a plot summary and a set of candidate genres, we can evaluate how well zero-shot learning performs for text classification tasks. This project demonstrates the flexibility of modern NLP models and shows how high-level APIs can be used to solve real-world problems with minimal setup.


In [1]:
# Install the transformers library
!pip install transformers

This cell installs the `transformers` library, which provides pre-trained models and pipelines used for zero-shot text classification. Installing it ensures that all required tools are available before building and running the movie genre classifier.


In [2]:
from transformers import pipeline

# Initialize the zero-shot classification pipeline
# This model is specifically trained to relate text to unseen labels
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In this cell, we initialize a zero-shot classification pipeline using the `facebook/bart-large-mnli` model. This model is trained to understand the relationship between text and labels, allowing it to classify movie plot summaries into genres without requiring any task-specific training.


In [7]:
# Movie plot summaries for testing
plots = [
    "A group of friends travel to a remote cabin in the woods where they are hunted by an ancient evil.",
    "A bumbling detective accidentally solves a high-stakes jewelry heist while trying to find his lost cat.",
    "A retired special forces operative must rescue his daughter from a high-security skyscraper taken over by terrorists."
]

# The candidate labels (genres)
candidate_labels = ["Horror", "Comedy", "Action"]

This cell defines a small set of movie plot summaries to test the genre classifier. Each plot represents a different style of story, and the candidate labels specify the possible genres the model can choose from: Horror, Comedy, and Action.


In [9]:
print("--- Movie Genre Classification Results ---")

for plot in plots:
    result = classifier(plot, candidate_labels)

    top_label = result['labels'][0]
    top_score = result['scores'][0]

    print(f"\nPlot: {plot[:60]}...")
    print(f"Predicted Genre: {top_label} (Confidence: {top_score:.4f})")

--- Movie Genre Classification Results ---

Plot: A group of friends travel to a remote cabin in the woods whe...
Predicted Genre: Action (Confidence: 0.4941)

Plot: A bumbling detective accidentally solves a high-stakes jewel...
Predicted Genre: Action (Confidence: 0.6548)

Plot: A retired special forces operative must rescue his daughter ...
Predicted Genre: Action (Confidence: 0.7648)


This cell runs the initial zero-shot genre classification using simple genre labels and the default single-label setting. For each plot summary, the model is forced to choose the single most likely genre from the provided options and returns a confidence score.

In this case, the model predicts **Action** for all three plots. This happens because zero-shot classification relies on semantic similarity between the text and the labels. Broad labels like “Action” can overlap with many plot elements (such as conflict, stakes, or movement), causing the model to over-prefer them. Since the model must choose only one label, it selects the genre that appears most generally applicable, even when other genres may be more appropriate.


In [10]:
from transformers import pipeline

# Use the larger model for better reasoning
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

plots = [
    "A group of friends travel to a remote cabin in the woods where they are hunted by an ancient evil.",
    "A bumbling detective accidentally solves a high-stakes jewelry heist while trying to find his lost cat.",
    "A retired special forces operative must rescue his daughter from a high-security skyscraper taken over by terrorists."
]

# Adding descriptive labels can sometimes help the model "see" the genre better
candidate_labels = ["horror thriller", "slapstick comedy", "action adventure"]
template = "This film is characterized as {}."

print("--- Improved Movie Genre Results ---")
for plot in plots:
    result = classifier(plot, candidate_labels, hypothesis_template=template)
    print(f"\nPlot: {plot[:50]}...")
    print(f"Top Genre: {result['labels'][0]} ({result['scores'][0]:.2%})")

Device set to use cpu


--- Improved Movie Genre Results ---

Plot: A group of friends travel to a remote cabin in the...
Top Genre: horror thriller (83.39%)

Plot: A bumbling detective accidentally solves a high-st...
Top Genre: action adventure (74.82%)

Plot: A retired special forces operative must rescue his...
Top Genre: action adventure (86.36%)


In this version of the experiment, the zero-shot classifier is improved by using more descriptive genre labels and a custom hypothesis template. Instead of short, ambiguous labels like “Action” or “Comedy,” the labels now include additional context (for example, “horror thriller” or “action adventure”), which helps the model better understand the intended meaning of each category.

The hypothesis template explicitly frames the classification task as a genre description, guiding the model to compare the plot summary against each label in a more structured way. As a result, the predictions become more confident and better aligned with the actual content of the plots. This demonstrates how zero-shot classification is highly sensitive to label wording and prompt design, and how small changes can significantly impact model behavior.


In [11]:
from transformers import pipeline

# Initializing the pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

plots = [
    "A group of friends travel to a remote cabin in the woods where they are hunted by an ancient evil.",
    "A bumbling detective accidentally solves a high-stakes jewelry heist while trying to find his lost cat.",
    "A retired special forces operative must rescue his daughter from a high-security skyscraper taken over by terrorists."
]

# Using clearer, distinct labels to reduce semantic overlap
candidate_labels = ["Horror", "Comedy", "Action"]
template = "This movie is a {} film."

print("--- Final Movie Genre Results (Multi-Label) ---")
for plot in plots:
    # Setting multi_label=True allows each genre score to be independent (0 to 1)
    result = classifier(plot, candidate_labels, hypothesis_template=template, multi_label=True)

    print(f"\nPlot: {plot[:60]}...")
    # Showing all labels that pass a 50% confidence threshold
    for label, score in zip(result['labels'], result['scores']):
        if score > 0.5:
            print(f"Detected Genre: {label} ({score:.2%})")

Device set to use cpu


--- Final Movie Genre Results (Multi-Label) ---

Plot: A group of friends travel to a remote cabin in the woods whe...
Detected Genre: Horror (96.38%)

Plot: A bumbling detective accidentally solves a high-stakes jewel...
Detected Genre: Comedy (54.23%)

Plot: A retired special forces operative must rescue his daughter ...
Detected Genre: Action (93.97%)


In this final approach, the zero-shot classifier is run in multi-label mode, which allows each genre to be evaluated independently instead of forcing the model to choose only one. By setting `multi_label=True`, the model assigns a separate confidence score to each genre, indicating how well it matches the plot.

This fixes the earlier issue where the model repeatedly predicted “Action” for all plots. Rather than comparing genres against each other, the model now checks whether each genre applies on its own. With clearer labels and a structured hypothesis template, the predictions become more accurate and easier to interpret, correctly identifying Horror, Comedy, and Action for the respective plots.



## Project Summary

In this project, we built a movie genre classifier using zero-shot learning to categorize plot summaries into Horror, Comedy, or Action. Instead of training a custom model, we used a pre-trained transformer model to infer genres based on semantic understanding.

We first tested a basic zero-shot setup with simple genre labels, which resulted in biased predictions where the model frequently chose “Action” due to semantic overlap between labels. To address this, we experimented with more descriptive labels and a custom hypothesis template, which improved confidence and alignment but still showed limitations.

Finally, we enabled multi-label classification, allowing each genre to be evaluated independently. This approach resolved the earlier issues and produced accurate, high-confidence predictions for all test plots. Overall, the project demonstrates how label design, prompt structure, and classification strategy significantly impact zero-shot model performance.
